### Libraries 

In [1]:
%matplotlib inline
from keras.applications.imagenet_utils import preprocess_input
from keras.backend.tensorflow_backend import set_session
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Concatenate
import numpy as np
from scipy.misc import imread
import tensorflow as tf
import pickle
import keras
from ssd import SSD300
from ssd_training import MultiboxLoss
from ssd_utils import BBoxUtility
from random import shuffle
import matplotlib.pyplot as plt
from SSD_tester import calc_detection_prec_rec, calc_detection_ap


np.set_printoptions(suppress=True)
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.45
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
voc_classes = ['Aeroplane', 'Bicycle', 'Bird', 'Boat', 'Bottle',
               'Bus', 'Car', 'Cat', 'Chair', 'Cow', 'Diningtable',
               'Dog', 'Horse','Motorbike', 'Person', 'Pottedplant',
               'Sheep', 'Sofa', 'Train', 'Tvmonitor']
NUM_CLASSES = len(voc_classes) + 1
input_shape = (300, 300, 3) #channel last

### Load the weight from the hdf5

In [3]:
gt = pickle.load(open('../pkls/VOC2007.pkl', 'rb'))
keys = sorted(gt.keys())
num_train = int(round(0.8 * len(keys)))
train_keys = keys[:num_train]
val_keys = keys[num_train:]
num_val = len(val_keys)

path_prefix = '../dataset/VOCdevkit/VOC2007/JPEGImages/'
# inputs = []
# images = []
# img_path = path_prefix + sorted(val_keys)[10]
# img = image.load_img(img_path, target_size=(300, 300))
# img = image.img_to_array(img)
# images.append(imread(img_path))
# inputs.append(img.copy())
# inputs = preprocess_input(np.array(inputs))


In [40]:
inputs = []
images = []
val_keys = sorted(val_keys)
for key in val_keys:
    img_path = path_prefix + key
    img = image.load_img(img_path, target_size=(300, 300))
    img = image.img_to_array(img)
    images.append(imread(img_path))
    inputs.append(img.copy())
inputs = preprocess_input(np.array(inputs))



priors = pickle.load(open('../pkls/prior_boxes_ssd300.pkl', 'rb'))
bbox_util = BBoxUtility(NUM_CLASSES, priors)
model = SSD300(input_shape, num_classes=NUM_CLASSES)
model.load_weights('../checkpoints/vocdevkit/weights.14-2.02.hdf5', by_name=True)
preds = model.predict(inputs, batch_size=32, verbose=1)
results = bbox_util.detection_out(preds, confidence_threshold=0.6)
results = np.array(results)




(64,)
990/990 [==============================] - 22s    


In [37]:
gt_bboxes = []
gt_labels = []
gt_scores = []
for key in val_keys:
    index = np.where(gt[key][:, 4:] == 1)
    gt_bboxes.append(gt[key][:, :4])
    gt_labels.append((index[1]).reshape(len(index[1]), 1))
    gt_scores.append(np.ones((len(index[1]), 1)))
gt_bboxes = np.array(gt_bboxes)
gt_labels = np.array(gt_labels)

pred_labels = []
pred_scores = []
pred_bboxes = []
for result in results:
    if len(result) != 0:
        nm = len(result[:, 1])
        pred_labels.append((result[:, 0] -1).reshape(nm, 1))
        pred_scores.append(result[:, 1:2].reshape(nm, 1))
        pred_bboxes.append(result[:, 2:].reshape(nm, 4))
    else:
        pred_labels.append(np.array([]).reshape(0, 1))
        pred_scores.append(np.array([]).reshape(0, 1))
        pred_bboxes.append(np.array([]).reshape(0, 1))
pred_labels = np.array(pred_labels)
pred_scores = np.array(pred_scores)

pred_bboxes = np.array(pred_bboxes)

In [38]:

prec, rec = calc_detection_prec_rec(pred_labels, pred_scores, pred_bboxes, gt_bboxes, gt_labels, iou_thresh=0.5)
print(rec)
    

[array([ 0.01408451,  0.02816901,  0.04225352,  0.05633803,  0.07042254,
        0.08450704,  0.09859155,  0.11267606,  0.12676056,  0.14084507,
        0.15492958,  0.16901408,  0.18309859,  0.1971831 ,  0.21126761,
        0.22535211,  0.23943662,  0.25352113,  0.26760563,  0.28169014,
        0.29577465,  0.30985915,  0.32394366,  0.33802817,  0.35211268,
        0.36619718,  0.38028169,  0.3943662 ,  0.4084507 ,  0.42253521,
        0.43661972,  0.45070423,  0.46478873,  0.47887324,  0.49295775,
        0.50704225,  0.52112676,  0.53521127,  0.54929577,  0.56338028,
        0.57746479,  0.5915493 ,  0.6056338 ,  0.61971831,  0.63380282,
        0.64788732,  0.66197183,  0.67605634,  0.69014085,  0.70422535,
        0.71830986,  0.73239437,  0.74647887,  0.76056338,  0.77464789,
        0.78873239,  0.8028169 ,  0.81690141,  0.83098592,  0.84507042,
        0.85915493,  0.87323944,  0.88732394,  0.90140845,  0.91549296,
        0.92957746,  0.94366197,  0.95774648,  0.97183099,  0.9

In [39]:
ap = calc_detection_ap(prec, rec, use_07_metric=True)
{'ap': ap, 'map': np.nanmean(ap)}

{'ap': array([ 1.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  0.90909091,  0.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ]),
 'map': 0.74545454545454548}

In [ ]:
preds = model.predict(inputs, batch_size=1, verbose=1)
results = bbox_util.detection_out(preds)

In [ ]:
for i, img in enumerate(images):
    # Parse the outputs.
    det_label = results[i][:, 0]
    det_conf = results[i][:, 1]
    det_xmin = results[i][:, 2]
    det_ymin = results[i][:, 3]
    det_xmax = results[i][:, 4]
    det_ymax = results[i][:, 5]

    # Get detections with confidence higher than 0.6.
    top_indices = [i for i, conf in enumerate(det_conf) if conf >= 0.8]

    top_conf = det_conf[top_indices]
    top_label_indices = det_label[top_indices].tolist()
    top_xmin = det_xmin[top_indices]
    top_ymin = det_ymin[top_indices]
    top_xmax = det_xmax[top_indices]
    top_ymax = det_ymax[top_indices]

    colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
    plt.imshow(img / 255.)
    currentAxis = plt.gca()

    for i in range(top_conf.shape[0]):
        xmin = int(round(top_xmin[i] * img.shape[1]))
        ymin = int(round(top_ymin[i] * img.shape[0]))
        xmax = int(round(top_xmax[i] * img.shape[1]))
        ymax = int(round(top_ymax[i] * img.shape[0]))
        score = top_conf[i]
        label = int(top_label_indices[i])
        label_name = voc_classes[label - 1]
        display_txt = '{:0.2f}, {}'.format(score, label_name)
        coords = (xmin, ymin), xmax-xmin+1, ymax-ymin+1
        color = colors[label]
        currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=color, linewidth=2))
        currentAxis.text(xmin, ymin, display_txt, bbox={'facecolor':color, 'alpha':0.5})
    
    plt.show()